First step is to read in the DNAm results (ss.csv) and compare the data to the microsatellites file. I'll need to clean up the microsatellites data so that I can join the two data together.

In [2]:
import pandas as pd
import numpy as np

In [3]:
# read in DNAm data sample sheet
ss = pd.read_csv("1-4_ss.csv")
ss.head(6)

,Sample_Name,Tissue,prob_snp_outlier,prob_snp_outlier_logodds,normalized_x_intensity,normalized_y_intensity,controls_failed,detp_05,Trophoblasts,Stromal,...,Flag_cell_p_IM,PC1_mbd500,PC2_mbd500,PC3_mbd500,PC4_mbd500,PC5_mbd500,PC1_mbd500_category,PC1_mbd500_outlier,Flag_EGA_adult,horvath_EGA
0,PL20_vc,Villi,0.051761,-4.884294,0.713571,0.957759,False,False,0.352232,0.181485,...,False,-0.018943,0.024996,-0.019521,-0.048586,-0.086354,+1sd < x < +2sd,False,False,0.3544
1,PL25_chc,Chorion,0.069870,-4.470582,0.832492,0.883894,False,False,0.055959,0.171550,...,NaN,0.006405,0.085893,-0.005294,-0.012090,-0.033918,NaN,NaN,False,NaN
2,PL78_chc,Chorion,0.062602,-4.620885,0.720464,0.920460,False,False,0.000000,0.532263,...,NaN,0.001000,0.102359,0.006611,-0.060718,-0.025342,NaN,NaN,False,NaN
3,PL56_chc,Chorion,0.047357,-4.825715,1.023450,0.262161,True,False,0.200416,0.238941,...,NaN,0.000776,0.075316,-0.008978,-0.024240,-0.042781,NaN,NaN,False,NaN
4,PL15_chc,Chorion,0.029213,-5.132586,0.716309,0.926453,False,False,0.112423,0.465669,...,NaN,-0.002935,0.110974,0.014890,-0.044844,-0.033285,NaN,NaN,False,NaN
5,PL91_chc,Chorion,0.037122,-4.951654,1.008799,0.247391,False,False,0.045411,0.540959,...,NaN,-0.002607,0.097883,0.009238,-0.060132,-0.033769,NaN,NaN,False,NaN


In [4]:
# read in microsatellite data
ms_data = pd.read_excel("2022-08-09 Contamination study result database_EA.xlsx")
ms_data.head(6)

,CASE ID,SAMPLE ID,MARKER TESTED,Marker size range,PCR CODE (note the one used in analysisONLY not the drop-outs),PCR date (yyyy-mm-dd),Analyzer Date (mm-dd-yy),ALELLE 1 size,ALELLE 2 size,ALELLE 3 size,...,ALELLE 1 area,ALELLE 2 area,ALELLE 3 area,ALELLE 4 area,Approx % contamination,result,coments,status,Unnamed: 19,Legend:
0,FT3v,FT3v,AR,~300bp,AE,2021-10-12,2021-10-13,269,272,NaN,...,UI,UI,NaN,NaN,0,Alleles too close for identification of possib...,possible female,pending; confirm other marker,NaN,Ammnion Chorion contamination study
1,NaN,FT3v,D7S3050,~150 bp,AP,2021-11-05,2021-11-09,145,149,154?,...,35293,30082,no selection,NaN,UI,one peak higher than the other and incalculabl...,NaN,"probable slight contamination; (1/3); noise, n...",NaN,NaN
2,NaN,FT3v,D7S1820,~250bp,AT,2021-11-26,2021-11-29,245,253?,262,...,95285,568,82475,NaN,0.318514,one peak higher than the other; small peak at ...,NaN,"probable slight contamination; (2/3); noise, n...",NaN,NaN
3,NaN,FT3v,D7S460,~175 bp,BO,2022-04-25,2022-04-25,177,185,NaN,...,52331,48473,NaN,NaN,0,noisy; but no evidence of contamination,NaN,no contamination,NaN,NaN
4,NaN,FT3v,D7S485,~250 bp,BR,2022-04-27,2022-04-28,241,243,NaN,...,41250,20209,NaN,NaN,0,no evidence of contamination,NaN,no contamination,NaN,NaN
5,FT22,FT22dec,AR,~300bp,AB,2021-09-23,2021-09-23,275,275,NaN,...,…,-,NaN,NaN,…,…,Overloaded Sample,pending; confirm other marker,NaN,NaN


The column SAMPLE ID will serve as a base to create the following columns:
Sample_Name should match the column in DNAm 
Tissue
Case id is missing values

In [6]:
# check matches between the two data:
sum(np.isin(ms_data["SAMPLE ID"], ss["Sample_Name"]))

0

In [7]:
ms_data["SAMPLE ID"].tolist()

['FT3v',
 'FT3v',
 'FT3v',
 'FT3v',
 'FT3v',
 'FT22dec',
 'FT22v',
 'FT22dec',
 'FT22v',
 'FT22dec',
 'FT22v',
 'FT22dec',
 'FT22v',
 'FT22dec',
 'FT22v',
 'FT26dec',
 'FT26v',
 'FT26dec',
 'FT26v',
 'FT26dec',
 'FT26v',
 'FT26dec',
 'FT26v',
 'FT26dec',
 'FT26v',
 'FT26dec',
 'FT26v',
 'FT27v',
 'FT27v',
 'FT27v',
 'FT27v',
 'FT28dec',
 'FT28vc',
 'FT28dec',
 'FT28vc',
 'FT28dec',
 'FT28vc',
 'FT28dec',
 'FT28vc',
 'FT28dec',
 'FT28vc',
 'FT28dec',
 'FT28vc',
 'FT38dec',
 'FT38vc',
 'FT38dec',
 'FT38vc',
 'FT38dec',
 'FT38vc',
 'FT38dec',
 'FT38vc',
 'FT38dec',
 'FT38vc',
 'FT38dec',
 'FT38vc',
 'FT39dec',
 'FT39vc',
 'FT39dec',
 'FT39vc',
 'FT39dec',
 'FT39vc',
 'FT41dec',
 'FT41vc (v1+v2)',
 'FT41vc (v3+v4)',
 'FT41vc',
 'FT41dec',
 'FT41vc (v1+v2)',
 'FT41vc (v3+v4)',
 'FT41vc',
 'FT41dec',
 'FT41vc (v1+v2)',
 'FT41vc (v3+v4)',
 'FT41vc',
 'FT41dec',
 'FT41vc (v1+v2)',
 'FT41vc (v3+v4)',
 'FT41vc',
 'FT41dec',
 'FT41vc (v1+v2)',
 'FT41vc (v3+v4)',
 'FT41vc',
 'FT41dec',
 'FT41vc (v

# Fix up sample_name column
This is important for joining

In [8]:
d = ms_data #working copy of data

# add underscore between sample id and tissue
d["Sample Name"] = d["SAMPLE ID"].replace('(\\d)\s*(\\D)', '\\1_\\2', regex = True)

#inspect change
d[["Sample Name", "SAMPLE ID"]]

,Sample Name,SAMPLE ID
0,FT3_v,FT3v
1,FT3_v,FT3v
2,FT3_v,FT3v
3,FT3_v,FT3v
4,FT3_v,FT3v
...,...,...
424,PM375_vc,PM375 vc
425,PM375_vm,PM375 vm
426,PM376_matwb,PM376 matwb
427,PM376_vc,PM376 vc


In [14]:
#remove everything after and including the whitespace
d["Sample Name"] = (d["Sample Name"]
                    .replace(' .*', '', regex=True))

# inspect change
d[["Sample Name", "SAMPLE ID"]][d["SAMPLE ID"].str.contains(' .*')]

,Sample Name,SAMPLE ID
62,FT41_vc,FT41vc (v1+v2)
63,FT41_vc,FT41vc (v3+v4)
66,FT41_vc,FT41vc (v1+v2)
67,FT41_vc,FT41vc (v3+v4)
70,FT41_vc,FT41vc (v1+v2)
...,...,...
424,PM375_vc,PM375 vc
425,PM375_vm,PM375 vm
426,PM376_matwb,PM376 matwb
427,PM376_vc,PM376 vc


In [15]:
#remove everything after the *
d["Sample Name"] = (d["Sample Name"]
                    .replace('\\*', '', regex=True))

# inspect change
d[["Sample Name", "SAMPLE ID"]][d["SAMPLE ID"].str.contains('\\*')]

,Sample Name,SAMPLE ID
168,NTD3_vc,NTD3vc*
171,NTD3_vc,NTD3vc*
174,NTD3_vc,NTD3vc*
177,NTD9_vc,NTD9vc*
180,NTD9_vc,NTD9vc*
183,NTD9_vc,NTD9vc*
186,NTD9_vc,NTD9vc*
189,NTD9_vc,NTD9vc*
192,NTD9_vc,NTD9vc*
195,NTD14_vc,NTD14vc* (v1+v2)


In [23]:
# this is a check function
sum(np.isin(d["Sample Name"], ss["Sample_Name"]))

113

In [29]:
d[["Sample Name"]] [~np.isin(d["Sample Name"], ss["Sample_Name"])]

,Sample Name
5,FT22_dec
7,FT22_dec
9,FT22_dec
11,FT22_dec
13,FT22_dec
...,...
424,PM375_vc
425,PM375_vm
426,PM376_matwb
427,PM376_vc


In [32]:
sum(np.isin(ss["Sample_Name"], d["Sample Name"]))

22

In [33]:
ss

,Sample_Name,Tissue,prob_snp_outlier,prob_snp_outlier_logodds,normalized_x_intensity,normalized_y_intensity,controls_failed,detp_05,Trophoblasts,Stromal,...,Flag_cell_p_IM,PC1_mbd500,PC2_mbd500,PC3_mbd500,PC4_mbd500,PC5_mbd500,PC1_mbd500_category,PC1_mbd500_outlier,Flag_EGA_adult,horvath_EGA
0,PL20_vc,Villi,0.051761,-4.884294,0.713571,0.957759,False,False,0.352232,0.181485,...,False,-0.018943,0.024996,-0.019521,-0.048586,-0.086354,+1sd < x < +2sd,False,False,0.354400
1,PL25_chc,Chorion,0.069870,-4.470582,0.832492,0.883894,False,False,0.055959,0.171550,...,NaN,0.006405,0.085893,-0.005294,-0.012090,-0.033918,NaN,NaN,False,NaN
2,PL78_chc,Chorion,0.062602,-4.620885,0.720464,0.920460,False,False,0.000000,0.532263,...,NaN,0.001000,0.102359,0.006611,-0.060718,-0.025342,NaN,NaN,False,NaN
3,PL56_chc,Chorion,0.047357,-4.825715,1.023450,0.262161,True,False,0.200416,0.238941,...,NaN,0.000776,0.075316,-0.008978,-0.024240,-0.042781,NaN,NaN,False,NaN
4,PL15_chc,Chorion,0.029213,-5.132586,0.716309,0.926453,False,False,0.112423,0.465669,...,NaN,-0.002935,0.110974,0.014890,-0.044844,-0.033285,NaN,NaN,False,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1673,GSM3109408,decidua,0.096707,-4.014660,0.946192,0.209675,True,False,0.298058,0.000000,...,NaN,0.007034,0.047750,-0.038119,0.022692,-0.036447,NaN,NaN,False,5.645992
1674,GSM3109410,decidua,0.058723,-4.527517,1.068022,0.255515,True,False,0.086960,0.163903,...,NaN,0.010982,0.084394,0.001102,0.041632,-0.019301,NaN,NaN,False,29.196195
1675,GSM3109411,decidua,0.041952,-4.805962,1.058808,0.241208,True,False,0.090643,0.126070,...,NaN,0.012149,0.076525,-0.012230,0.041021,-0.008042,NaN,NaN,False,45.005058
1676,GSM3109412,decidua,0.048539,-4.676530,1.068607,0.232210,True,False,0.084874,0.177034,...,NaN,0.011284,0.082315,-0.003955,0.044421,-0.014730,NaN,NaN,False,32.798708


In [ ]:
# put it all together
d["SAMPLE ID"] = (d["SAMPLE ID"]
     .replace('(\\d)\s*(\\D)', '\\1_\\2', regex = True)
     .replace(' .*', '', regex=True)
    )
d